+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-

# Greeks and Positions

In [18]:
import requests
import csv
import time
import json
import pandas as pd
import re
from datetime import datetime

BASE_URL = "https://srm-api.sterlingtradingtech.com/pts-risk"
USERNAME = "AOSScalp"
PASSWORD = "Scalp1"
mapping = {
    "3OB05285": "12P52852",
    "3OB05287": "14P52874",
    "3OB05259": "15P52595",
    "3OB05283": "16P52836",
    "3OB05286": "17P52867",
    "3OB05294": "18P529422",
    "3OB05295": "19P529523",
    "3OB05255": "1P525501",
    "3OB05292": "20P529221",
    "3OB05303": "21P530324",
    "3OB05306": "22P530624",
    "3OB05305": "23P530525",
    "3OB05311": "24P531126",
    "3OB05314": "25P531426",
    "3OB05315": "26P531527",
    "3OB05322": "27P532248",
    "3OB05323": "28P532349",
    "3OB05324": "29P532451",
    "3OB05258": "2P525802",
    "3OB05326": "30P532652",
    "3OB05327": "31P532753",
    "3OB05345": "32P534563",
    "3OB05346": "33P534612",
    "3OB05347": "34P534728",
    "3OB05360": "35P536046",
    "3OB05361": "36P536158",
    "3OB05362": "37P536299",
    "3OB05363": "38P536381",
    "3OB05364": "39P536458",
    "3OB05284": "3P528403",
    "3OB05365": "40P536571",
    "3OB05366": "41P536622",
    "3OB05367": "42P536787",
    "3OB05256": "4P525604",
    "3OB05257": "5P525705",
    "3OB05288": "6P528806",
    "3OB05260": "7P526007",
    "3OB05289": "8P528918",
    "3OB05261": "9P526109",
    "3OB05254": "SIPM5254"
}

def get_access_token():
    url = f"{BASE_URL}/oauth/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": USERNAME,
        "client_secret": PASSWORD
    }
    response = requests.post(url, data=payload)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to obtain access token. Status code: {response.status_code}")

def call_api(endpoint, access_token, json_body):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    url = f"{BASE_URL}{endpoint}"
    
    response = requests.post(url, headers=headers, json=json_body)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed. Status code: {response.status_code}")

def var_main(access_token):
    true = True
    var_json_body = {
        "accounts": ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"],
        "correlator": "correlator",
        "detailAnswer": true,
        "messageType": "Snapshot"
    }
    var_data = call_api("/service/var", access_token, var_json_body)
    return var_data

def greek_main(access_token, symbols_list, shockLevel, underlyingShock, volativityShock):
    greek_json_body = {
        "correlator": "test",
        "instruments": symbols_list,
        "messageType": "Snapshot",
        "shocks": [
            {
            "shockLevel": shockLevel,
            "underlyingShock": underlyingShock,
            "volatilityShock": volativityShock
            }
        ],
    }
    greek_data = call_api("/service/RiskGatewaySnapshot", access_token, greek_json_body)
    return greek_data

def positions_main(access_token):
    positions_json_body = ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"]
    
    # Replace 'your_endpoint' with the desired endpoint from the STT API documentation
    postitions_data = call_api("/service/account-positions", access_token, positions_json_body)
    print("API call result:", postitions_data)
    return postitions_data

def house_main(access_token):
    house_json_body = ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"]
    house_data = call_api("/service/house-policy", access_token, house_json_body)
    print("API call result:", house_data)
    return house_data

def clean_symbol(symbol):
    symbol = symbol.replace(" ", "")  # Remove all spaces
    if symbol.startswith("."):
        underlying = re.match(r'[^\d]+', symbol).group()
        underlying = underlying.replace(".", "")  # Remove the period from the underlying variable
        return symbol, underlying
    else:
        return None, symbol

def format_symbol(symbol):
    # Remove the decimal in the symbol
    symbol_formatted = symbol.replace('.', '')
    # Replace multiple spaces with a single space
    symbol_formatted = ' '.join(symbol_formatted.split())
    # Remove the space between the second set of numerical values and the "C" or "P"
    symbol_formatted = re.sub(r"([CP])(\s+)(\d+)", r"\1\3", symbol_formatted)
    # Add the period at the beginning
    symbol_formatted = '.' + symbol_formatted

    # assuming the underlying asset is determined by the part of the symbol before the first space
    underlying = symbol.split()[0]

    return symbol_formatted, underlying


def account_pos_run():
    access_token = get_access_token()
    print("Access token obtained:", access_token)

    positions = positions_main(access_token)

    rows = []
    for item in positions:
        account = item["account"]
        accountName = item["accountName"]
        for position in item["positions"]:
            row = {
                "account": account,
                "accountName": accountName,
                "symbol": position["symbol"],
                "pos": position["pos"]
            }
            rows.append(row)

    # Create a DataFrame from the rows list
    df = pd.DataFrame(rows)

    # Assuming your DataFrame is called df
    df['contract_name'], df['underlying'] = zip(*df['symbol'].apply(format_symbol))

    # Create a new DataFrame that contains the 'symbol', 'account', and 'pos' columns
    symbol_account_df = df[['symbol', 'account', 'pos']].drop_duplicates()

    # Create a list of unique symbols from the 'symbol' column
    symbols_list = df['symbol'].to_list()

    greek_data = greek_main(access_token, symbols_list, 'MarketShock', -100, 100)

    # Extract the 'results' list from the JSON data and create a DataFrame
    results_df = pd.DataFrame(greek_data['results'])

    # Merge the results_df with symbol_account_df on the 'symbol' column
    results_df_with_account_pos = results_df.merge(symbol_account_df, on='symbol')

    #results_df_with_account_pos.to_csv("greeks_df_with_account_pos.csv")
    return results_df_with_account_pos[
        [
            "account", "symbol", "pos", "optionPrice", "underlyingPrice",
            "shockedUnderlyingPrice", "volatility", "volatilityShock", "shockedVolatility",
            "theo", "delta", "gamma", "vega", "theta", "rho", "useClosePrice", "lastUpdate",
            "shockLevel"
        ]
    ]

def greeks_accounts_pos_df():
    # Run the script and write the initial data to the CSV file
    account_pos_df = account_pos_run()
    account_pos_df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Multiply each Greek value by the position and store the results in new columns
    account_pos_df['Theo Position'] = account_pos_df['theo'] * account_pos_df['pos']
    account_pos_df['Delta Position'] = account_pos_df['delta'] * account_pos_df['pos']
    account_pos_df['Gamma Position'] = account_pos_df['gamma'] * account_pos_df['pos']
    account_pos_df['Vega Position'] = account_pos_df['vega'] * account_pos_df['pos']
    account_pos_df['Theta Position'] = account_pos_df['theta'] * account_pos_df['pos']
    account_pos_df['Rho Position'] = account_pos_df['rho'] * account_pos_df['pos']

    new_column_order = [
        "timestamp", "lastUpdate", "account", "symbol", "pos",
        "optionPrice", "underlyingPrice", "shockedUnderlyingPrice",
        "volatility", "volatilityShock", "shockedVolatility",
        "theo", "delta", "gamma", "vega", "theta", "rho",
        "Theo Position", "Delta Position", "Gamma Position",
        "Vega Position", "Theta Position", "Rho Position"
    ]
    data_frame = account_pos_df[new_column_order]

    #Lower case the column names
    data_frame.columns = data_frame.columns.str.lower()
    return data_frame

greeks_accounts_pos_df = greeks_accounts_pos_df()
greeks_accounts_pos_df

Access token obtained: 61f8bf6b-707d-4be6-8033-a0d53aee2d5d
API call result: [{'account': '3OB05311', 'accountName': '3OB05311-24P', 'positions': [{'symbol': '.AI 231020P17500', 'pos': 0}, {'symbol': '.KRTX 240119P200000', 'pos': 0}, {'symbol': '.MA 230721P350000', 'pos': 0}, {'symbol': '.MA 230721C380000', 'pos': 0}, {'symbol': '.IBRX 231020C2500', 'pos': 0}, {'symbol': '.SWKS 240119C110000', 'pos': 0}, {'symbol': '.AVGO 230915P630000', 'pos': 0}, {'symbol': '.AXSM 230526C68000', 'pos': 0}, {'symbol': '.TSLA 250117C230000', 'pos': 0}, {'symbol': '.AI 231020P30000', 'pos': 0}, {'symbol': '.IBRX 231020P2500', 'pos': 0}, {'symbol': '.AVGO 230915P700000', 'pos': 0}, {'symbol': '.TSLA 250620C250000', 'pos': 0}, {'symbol': '.BKNG 230623C2920000', 'pos': 0}, {'symbol': '.BKNG 230623C2910000', 'pos': 0}, {'symbol': '.AXSM 230616C60000', 'pos': 0}, {'symbol': '.SWKS 250117C135000', 'pos': 0}, {'symbol': '.KRTX 250117P190000', 'pos': 0}]}, {'account': '3OB05256', 'accountName': '3OB05256-4P', '

,timestamp,lastupdate,account,symbol,pos,optionprice,underlyingprice,shockedunderlyingprice,volatility,volatilityshock,...,gamma,vega,theta,rho,theo position,delta position,gamma position,vega position,theta position,rho position
0,2023-05-23 13:16:39,292269055-12-02 10:47:04,3OB05255,PBRA,2600,0.00000,0.000,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0
1,2023-05-23 13:16:39,2023-05-23 13:15:31,3OB05258,.HZNP 230818P100000,0,2.90000,100.365,0.100365,0.183250,100,...,-0.000485,0.000000,0.000000,0.000000,0.00000,-0.000000,-0.000000,0.0,0.0,0.0
2,2023-05-23 13:16:39,2023-05-23 13:15:31,3OB05366,.HZNP 230818P100000,0,2.90000,100.365,0.100365,0.183250,100,...,-0.000485,0.000000,0.000000,0.000000,0.00000,-0.000000,-0.000000,0.0,0.0,0.0
3,2023-05-23 13:16:39,2023-05-23 13:03:28,3OB05284,ADBE,0,0.00000,371.435,0.371435,0.000000,100,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0
4,2023-05-23 13:16:39,2023-05-23 13:12:13,3OB05315,.HUM 230721P410000,0,0.80512,505.415,0.505415,0.300823,100,...,-0.000118,0.000000,0.000000,0.000000,0.00000,-0.000000,-0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,2023-05-23 13:16:39,2023-05-23 13:15:39,3OB05306,.SPXW 230915C4110000,0,215.25000,4158.420,4.158420,0.178498,100,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0
2180,2023-05-23 13:16:39,2023-05-23 13:05:35,3OB05367,.MRK 240119C130000,0,2.74000,113.735,0.113735,0.202057,100,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0
2181,2023-05-23 13:16:39,2023-05-23 13:16:17,3OB05326,.KSS 250117C12500,0,8.15000,19.455,0.019455,0.559374,100,...,0.148681,0.000049,-0.000004,0.000021,0.00000,0.000000,0.000000,0.0,-0.0,0.0
2182,2023-05-23 13:16:39,2023-05-23 13:16:10,3OB05256,.REGN 240119P290000,-9,1.25000,751.080,0.751080,0.572516,100,...,-0.000167,0.000000,0.000000,0.000000,-2603.24028,8.999991,0.001503,-0.0,-0.0,-0.0


In [ ]:
def positions_main(access_token):
    positions_json_body = ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"]
    
    # Replace 'your_endpoint' with the desired endpoint from the STT API documentation
    postitions_data = call_api("/service/account-positions", access_token, positions_json_body)
    print("API call result:", postitions_data)
    return postitions_data

# STT Report

In [ ]:
import requests
import csv
import time
import json
import pandas as pd

BASE_URL = "https://srm-api.sterlingtradingtech.com/pts-risk"
USERNAME = "AOSScalp"
PASSWORD = "Scalp1"
mapping = {
        "3OB05285": "12P52852",
        "3OB05287": "14P52874",
        "3OB05259": "15P52595",
        "3OB05283": "16P52836",
        "3OB05286": "17P52867",
        "3OB05294": "18P529422",
        "3OB05295": "19P529523",
        "3OB05255": "1P525501",
        "3OB05292": "20P529221",
        "3OB05303": "21P530324",
        "3OB05306": "22P530624",
        "3OB05305": "23P530525",
        "3OB05311": "24P531126",
        "3OB05314": "25P531426",
        "3OB05315": "26P531527",
        "3OB05322": "27P532248",
        "3OB05323": "28P532349",
        "3OB05324": "29P532451",
        "3OB05258": "2P525802",
        "3OB05326": "30P532652",
        "3OB05327": "31P532753",
        "3OB05345": "32P534563",
        "3OB05346": "33P534612",
        "3OB05347": "34P534728",
        "3OB05360": "35P536046",
        "3OB05361": "36P536158",
        "3OB05362": "37P536299",
        "3OB05363": "38P536381",
        "3OB05364": "39P536458",
        "3OB05284": "3P528403",
        "3OB05365": "40P536571",
        "3OB05366": "41P536622",
        "3OB05367": "42P536787",
        "3OB05256": "4P525604",
        "3OB05257": "5P525705",
        "3OB05288": "6P528806",
        "3OB05260": "7P526007",
        "3OB05289": "8P528918",
        "3OB05261": "9P526109",
        "3OB05254": "SIPM5254"
    }

def get_access_token():
    url = f"{BASE_URL}/oauth/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": USERNAME,
        "client_secret": PASSWORD
    }
    response = requests.post(url, data=payload)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to obtain access token. Status code: {response.status_code}")

def house_main(access_token):
    true = True
    house_json_body = ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"]

    house_data = call_api("/service/house-policy", access_token, house_json_body)
    return house_data

def positions_main(access_token):
    positions_json_body = ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"]
    
    positions_data = call_api("/service/account-positions", access_token, positions_json_body)
    return positions_data


def call_api(endpoint, access_token, json_body):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    url = f"{BASE_URL}{endpoint}"
    
    response = requests.post(url, headers=headers, json=json_body)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed. Status code: {response.status_code}")
    return response

def get_house_data():
    access_token = get_access_token()
    print("Access token obtained:", access_token)
    
    # Suppose your json is stored in 'house_data' variable
    house_data = house_main(access_token)
    print(f"House data obtained: {house_data}")

    data = []

    # Loop through the JSON object
    for entry in house_data:
        # Extract the necessary values
        account = entry["account"]
        account_name = entry["accountName"]
        smv = entry["smv"]
        sod_equity = entry["sodEquity"]
        sod_house_excess = entry["sodHouseExcess"]
        tims = entry["tims"]
        tims_addon = entry["timsAddon"]
        equity = entry["equity"]
        
        # Append to the data list
        data.append([account, account_name, smv, sod_equity, sod_house_excess, tims, tims_addon, equity])

    # Define the column names
    columns = ["account", "accountName", "smv", "sodEquity", "sodHouseExcess", "tims", "timsAddon", "equity"]

    # Create the dataframe
    df = pd.DataFrame(data, columns=columns)
    
    #df["account"] = df["account"].replace(mapping)
    #df = df.drop(columns=["accountName", "smv", "sodHouseExcess", "tims", "timsAddon", "equity"])
    return df

def get_position_data():
    access_token = get_access_token()
    print("Access token obtained:", access_token)

    positions = positions_main(access_token)
    df = json_normalize(positions, 'positions', ['account', 'accountName'])
    return df

def clean_symbol(symbol):
    symbol = symbol.replace(" ", "")  # Remove all spaces
    if not symbol.startswith("."):
        underlying_match = re.match(r'[^\d]+', symbol)
        if underlying_match is None:
            pass
            return symbol
        underlying = underlying_match.group()  # Extract the underlying symbol
        date_strike_match = re.search(r'(\d+)', symbol)
        if date_strike_match is None:
            pass
            return symbol
        date_strike = date_strike_match.group(0)  # Extract the date and strike
        option_type = symbol[-1]  # Extract the option type
        new_symbol = '.' + underlying + ' ' + date_strike[2:8] + option_type + date_strike[9:12] + '000'
        return new_symbol
    else:
        return symbol

def merge_positions_and_sod_data():
    account_sod_data = get_house_data()
    position_data = get_position_data()
    merged_df = pd.merge(position_data, account_sod_data, left_on='account', right_on='account', how='left')
    return merged_df

def var_main(access_token):
    true = True
    var_json_body = {
        "accounts": ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
                    "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
                    "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
                    "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
                    "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
                    "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
                    "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"],
        "correlator": "correlator",
        "detailAnswer": true,
        "messageType": "Snapshot"
    }
    var_data = call_var_api("/service/var", access_token, var_json_body)
    return var_data

def call_var_api(endpoint, access_token, json_body):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    url = f"{BASE_URL}{endpoint}"
    
    response = requests.post(url, headers=headers, json=json_body)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed. Status code: {response.status_code}")

def var_run():
    access_token = get_access_token()
    print("Access token obtained:", access_token)

    var_data = var_main(access_token)

    # Normalize the JSON data for varData
    var_data_df = pd.json_normalize(var_data['accountsResponse'], sep='.')
    var_data_df = var_data_df[['account', 'varData.var99', 'varData.cvar99', 'varData.var95', 'varData.cvar95', 'varData.var99Date', 'varData.var95Date']]

    # Normalize the JSON data for var99Contribution and var95Contribution with the 'account' column
    var99_contrib_df = pd.json_normalize(var_data['accountsResponse'], record_path=['varData', 'var99Contribution'], meta=['account'], sep='.')
    var95_contrib_df = pd.json_normalize(var_data['accountsResponse'], record_path=['varData', 'var95Contribution'], meta=['account'], sep='.')

    # Merge the two DataFrames on the 'symbol' and 'account' columns
    merged_df = var99_contrib_df.merge(var95_contrib_df, on=['symbol', 'account'], suffixes=('_var99', '_var95'))
    merged_df = merged_df[['account', 'symbol', 'result_var99', 'priceScenario_var99', 'ivScenario_var99', 'result_var95', 'priceScenario_var95', 'ivScenario_var95']]

    # Merge var_data_df with merged_df on the 'account' column
    final_df = var_data_df.merge(merged_df, on='account')

    final_df["account"] = final_df["account"].replace(mapping)
    final_df.to_csv("final_df.csv")
    print(final_df)
    return final_df

def sod_acc_sodpos_run():
    sod_acc_sodpos = merge_positions_and_sod_data()
    sod_acc_sodpos["account"] = sod_acc_sodpos["account"].replace(mapping)
    return sod_acc_sodpos

def combined_var_house_pos():
    sod_acc_sodpos_df = sod_acc_sodpos_run()
    var_df = var_run()
    var_df['symbol'] = var_df['symbol'].str.split(' ').str[1]

    # Converting 'account' and 'symbol' columns to string in both dataframes
    sod_acc_sodpos_df['account'] = sod_acc_sodpos_df['account'].astype(str)
    sod_acc_sodpos_df['symbol'] = sod_acc_sodpos_df['symbol'].astype(str)

    var_df['account'] = var_df['account'].astype(str)
    var_df['symbol'] = var_df['symbol'].astype(str)

    # Merge the dataframes on 'account' and 'symbol'
    merged_df = pd.merge(sod_acc_sodpos_df, var_df, on=['account', 'symbol'], how='outer')

    print(sod_acc_sodpos_df)
    print(var_df)
    return merged

combined_var_house_pos_df = combined_var_house_pos()
# First, create a new column 'clean_symbol' in greeks_accounts_pos_df that only contains values after "."
greeks_accounts_pos_df['clean_symbol'] = greeks_accounts_pos_df['symbol'].str.split(' ').str[0].str[1:]

# Now merge the two dataframes on 'account' and 'clean_symbol'
merged_df = pd.merge(combined_var_house_pos_df, greeks_accounts_pos_df, left_on=['account', 'symbol_x'], right_on=['account', 'clean_symbol'], how='outer')

merged_df.to_csv("STT_report.csv")

# House Data

In [6]:
import requests
import csv
import time
import json
import pandas as pd

BASE_URL = "https://srm-api.sterlingtradingtech.com/pts-risk"
USERNAME = "AOSScalp"
PASSWORD = "Scalp1"
mapping = {
        "3OB05285": "12P52852",
        "3OB05287": "14P52874",
        "3OB05259": "15P52595",
        "3OB05283": "16P52836",
        "3OB05286": "17P52867",
        "3OB05294": "18P529422",
        "3OB05295": "19P529523",
        "3OB05255": "1P525501",
        "3OB05292": "20P529221",
        "3OB05303": "21P530324",
        "3OB05306": "22P530624",
        "3OB05305": "23P530525",
        "3OB05311": "24P531126",
        "3OB05314": "25P531426",
        "3OB05315": "26P531527",
        "3OB05322": "27P532248",
        "3OB05323": "28P532349",
        "3OB05324": "29P532451",
        "3OB05258": "2P525802",
        "3OB05326": "30P532652",
        "3OB05327": "31P532753",
        "3OB05345": "32P534563",
        "3OB05346": "33P534612",
        "3OB05347": "34P534728",
        "3OB05360": "35P536046",
        "3OB05361": "36P536158",
        "3OB05362": "37P536299",
        "3OB05363": "38P536381",
        "3OB05364": "39P536458",
        "3OB05284": "3P528403",
        "3OB05365": "40P536571",
        "3OB05366": "41P536622",
        "3OB05367": "42P536787",
        "3OB05256": "4P525604",
        "3OB05257": "5P525705",
        "3OB05288": "6P528806",
        "3OB05260": "7P526007",
        "3OB05289": "8P528918",
        "3OB05261": "9P526109",
        "3OB05254": "SIPM5254"
    }

def call_api(endpoint, access_token, json_body):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    url = f"{BASE_URL}{endpoint}"
    
    response = requests.post(url, headers=headers, json=json_body)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed. Status code: {response.status_code}")

def get_access_token():
    url = f"{BASE_URL}/oauth/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": USERNAME,
        "client_secret": PASSWORD
    }
    response = requests.post(url, data=payload)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to obtain access token. Status code: {response.status_code}")

def house_main(access_token):
    true = True
    house_json_body = ["3OB05285", 
                       "3OB05287", 
                       "3OB05259", 
                       "3OB05283", 
                       "3OB05286", 
                       "3OB05294", 
                       "3OB05295", 
                       "3OB05255", 
                       "3OB05292", 
                       "3OB05303",
                       "3OB05306", 
                       "3OB05305",
                       "3OB05311",
                       "3OB05314",
                       "3OB05315",
                       "3OB05322",
                       "3OB05323", 
                       "3OB05324",
                       "3OB05258",
                       "3OB05326",
                       "3OB05327",
                       "3OB05345",
                       "3OB05346", 
                       "3OB05347", 
                       "3OB05360", 
                       "3OB05361", 
                       "3OB05362", 
                       "3OB05363", 
                       "3OB05364", 
                       "3OB05284", 
                       "3OB05365", 
                       "3OB05366", 
                       "3OB05367", 
                       "3OB05256", 
                       "3OB05257", 
                       "3OB05288", 
                       "3OB05260", 
                       "3OB05289", 
                       "3OB05261", 
                       "3OB0525"]


    house_data = call_api("/service/house-policy", access_token, house_json_body)
    return house_data

def get_house_data():
    access_token = get_access_token()
    print("Access token obtained:", access_token)
    
    # Suppose your json is stored in 'house_data' variable
    house_data = house_main(access_token)
    print(f"House data obtained: {house_data}")

    data = []

    # Loop through the JSON object
    for entry in house_data:
        # Extract the necessary values
        account = entry["account"]
        account_name = entry["accountName"]
        smv = entry["smv"]
        sod_equity = entry["sodEquity"]
        sod_house_excess = entry["sodHouseExcess"]
        tims = entry["tims"]
        tims_addon = entry["timsAddon"]
        equity = entry["equity"]
        
        # Append to the data list
        data.append([account, account_name, smv, sod_equity, sod_house_excess, tims, tims_addon, equity])

    # Define the column names
    columns = ["account", "accountName", "smv", "sodEquity", "sodHouseExcess", "tims", "timsAddon", "equity"]

    # Create the dataframe
    df = pd.DataFrame(data, columns=columns)
    
    #df["account"] = df["account"].replace(mapping)
    #df = df.drop(columns=["accountName", "smv", "sodHouseExcess", "tims", "timsAddon", "equity"])
    return df

house = get_house_data()
house

Access token obtained: 61f8bf6b-707d-4be6-8033-a0d53aee2d5d
House data obtained: [{'account': '3OB05286', 'accountName': '3OB05286-17P', 'housePolicyName': 'APEX', 'equity': 158768.67, 'sodEquity': 158625.67, 'pl': 143.0, 'lmv': 0.0, 'smv': 34540.0, 'tims': 2827.19, 'timsAddon': 2827.19, 'var': None, 'riskShockMethods': {'singleName': -29135.62, 'relativeValueStress': -10672.91, 'worst5': -15203.7, 'sectorStress': -1321.89, 'macroStress': -23042.57}, 'addonValues': {'multiplied': 0.0}, 'finalHouse': 29135.62, 'houseExcess': 129633.05, 'pmExcess': 155941.48, 'sodHouseExcess': 129490.05, 'riskDetails': None, 'timsDetails': None, 'requirementDetails': None}, {'account': '3OB05285', 'accountName': '3OB05285-12P', 'housePolicyName': 'APEX', 'equity': 266637.91, 'sodEquity': 263833.91, 'pl': 2804.0, 'lmv': 0.0, 'smv': 0.0, 'tims': 0.0, 'timsAddon': 0.0, 'var': None, 'riskShockMethods': {'singleName': 0.0, 'relativeValueStress': 0.0, 'worst5': 0.0, 'sectorStress': 0.0, 'macroStress': 0.0}, 'a

,account,accountName,smv,sodEquity,sodHouseExcess,tims,timsAddon,equity
0,3OB05286,3OB05286-17P,34540.00,158625.67,129490.05,2827.19,2827.19,158768.67
1,3OB05285,3OB05285-12P,0.00,263833.91,263833.91,0.00,0.00,266637.91
2,3OB05362,3OB05362-37P,0.00,-497.84,-682.84,156.06,156.06,-254.84
3,3OB05288,3OB05288-6P,0.00,-15703.21,-15735.71,32.50,32.50,-15350.19
4,3OB05365,3OB05365-40P,0.00,-276.60,-276.60,0.00,0.00,-423.60
5,3OB05287,3OB05287-14P,5105.50,80785.01,77821.71,2920.45,2920.45,81168.33
6,3OB05364,3OB05364-39P,501.39,-2364.61,-4120.15,477.36,477.36,-2064.00
7,3OB05260,3OB05260-7P,5760.00,446092.36,259149.65,112.50,112.50,445928.36
8,3OB05284,3OB05284-3P,0.00,222392.73,222392.73,0.00,0.00,222077.17
9,3OB05261,3OB05261-9P,287307.93,285577.85,236112.34,47714.71,47714.71,275009.42


# Var 99 & Var 99

In [7]:
import requests
import csv
import time
import json
import pandas as pd
import re
from datetime import datetime

BASE_URL = "https://srm-api.sterlingtradingtech.com/pts-risk"
USERNAME = "AOSScalp"
PASSWORD = "Scalp1"
mapping = {
    "3OB05285": "12P52852",
    "3OB05287": "14P52874",
    "3OB05259": "15P52595",
    "3OB05283": "16P52836",
    "3OB05286": "17P52867",
    "3OB05294": "18P529422",
    "3OB05295": "19P529523",
    "3OB05255": "1P525501",
    "3OB05292": "20P529221",
    "3OB05303": "21P530324",
    "3OB05306": "22P530624",
    "3OB05305": "23P530525",
    "3OB05311": "24P531126",
    "3OB05314": "25P531426",
    "3OB05315": "26P531527",
    "3OB05322": "27P532248",
    "3OB05323": "28P532349",
    "3OB05324": "29P532451",
    "3OB05258": "2P525802",
    "3OB05326": "30P532652",
    "3OB05327": "31P532753",
    "3OB05345": "32P534563",
    "3OB05346": "33P534612",
    "3OB05347": "34P534728",
    "3OB05360": "35P536046",
    "3OB05361": "36P536158",
    "3OB05362": "37P536299",
    "3OB05363": "38P536381",
    "3OB05364": "39P536458",
    "3OB05284": "3P528403",
    "3OB05365": "40P536571",
    "3OB05366": "41P536622",
    "3OB05367": "42P536787",
    "3OB05256": "4P525604",
    "3OB05257": "5P525705",
    "3OB05288": "6P528806",
    "3OB05260": "7P526007",
    "3OB05289": "8P528918",
    "3OB05261": "9P526109",
    "3OB05254": "SIPM5254"
}

def get_access_token():
    url = f"{BASE_URL}/oauth/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": USERNAME,
        "client_secret": PASSWORD
    }
    response = requests.post(url, data=payload)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to obtain access token. Status code: {response.status_code}")

def call_api(endpoint, access_token, json_body):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    url = f"{BASE_URL}{endpoint}"
    
    response = requests.post(url, headers=headers, json=json_body)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed. Status code: {response.status_code}")

def var_main(access_token):
    true = True
    var_json_body = {
        "accounts": ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"],
        "correlator": "correlator",
        "detailAnswer": true,
        "messageType": "Snapshot"
    }
    var_data = call_api("/service/var", access_token, var_json_body)
    return var_data

def var_run():
    access_token = get_access_token()
    print("Access token obtained:", access_token)

    var_data = var_main(access_token)

    # Normalize the JSON data for varData
    var_data_df = pd.json_normalize(var_data['accountsResponse'], sep='.')
    var_data_df = var_data_df[['account', 'varData.var99', 'varData.cvar99', 'varData.var95', 'varData.cvar95', 'varData.var99Date', 'varData.var95Date']]

    # Normalize the JSON data for var99Contribution and var95Contribution with the 'account' column
    var99_contrib_df = pd.json_normalize(var_data['accountsResponse'], record_path=['varData', 'var99Contribution'], meta=['account'], sep='.')
    var95_contrib_df = pd.json_normalize(var_data['accountsResponse'], record_path=['varData', 'var95Contribution'], meta=['account'], sep='.')

    # Merge the two DataFrames on the 'symbol' and 'account' columns
    merged_df = var99_contrib_df.merge(var95_contrib_df, on=['symbol', 'account'], suffixes=('_var99', '_var95'))
    merged_df = merged_df[['account', 'symbol', 'result_var99', 'priceScenario_var99', 'ivScenario_var99', 'result_var95', 'priceScenario_var95', 'ivScenario_var95']]

    # Merge var_data_df with merged_df on the 'account' column
    final_df = var_data_df.merge(merged_df, on='account')

    final_df["account"] = final_df["account"].replace(mapping)
    final_df.to_csv("final_df.csv")
    print(final_df)
    return final_df

var_data = var_run()
var_data

Access token obtained: 61f8bf6b-707d-4be6-8033-a0d53aee2d5d
       account  varData.var99  varData.cvar99  varData.var95  varData.cvar95  \
0    24P531126       0.000000        0.000000       0.000000         0.00000   
1    24P531126       0.000000        0.000000       0.000000         0.00000   
2    24P531126       0.000000        0.000000       0.000000         0.00000   
3    24P531126       0.000000        0.000000       0.000000         0.00000   
4    24P531126       0.000000        0.000000       0.000000         0.00000   
..         ...            ...             ...            ...             ...   
711  23P530525    -581.567956     -653.956425    -425.870245      -528.90911   
712  23P530525    -581.567956     -653.956425    -425.870245      -528.90911   
713  23P530525    -581.567956     -653.956425    -425.870245      -528.90911   
714  23P530525    -581.567956     -653.956425    -425.870245      -528.90911   
715  23P530525    -581.567956     -653.956425    -425.870245

,account,varData.var99,varData.cvar99,varData.var95,varData.cvar95,varData.var99Date,varData.var95Date,symbol,result_var99,priceScenario_var99,ivScenario_var99,result_var95,priceScenario_var95,ivScenario_var95
0,24P531126,0.000000,0.000000,0.000000,0.00000,2023-01-27,2023-01-12,AVGO,0.000000,-1.27,-0.36,0.000000,0.66,-7.38
1,24P531126,0.000000,0.000000,0.000000,0.00000,2023-01-27,2023-01-12,TSLA,0.000000,11.00,27.69,0.000000,0.28,-2.65
2,24P531126,0.000000,0.000000,0.000000,0.00000,2023-01-27,2023-01-12,MA,0.000000,-0.85,-4.69,0.000000,0.52,-5.46
3,24P531126,0.000000,0.000000,0.000000,0.00000,2023-01-27,2023-01-12,AI,0.000000,17.84,43.89,0.000000,1.26,-2.39
4,24P531126,0.000000,0.000000,0.000000,0.00000,2023-01-27,2023-01-12,KRTX,0.000000,0.41,-0.11,0.000000,4.44,-8.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,23P530525,-581.567956,-653.956425,-425.870245,-528.90911,2022-07-21,2022-07-27,AZO,0.000000,-2.54,2.09,0.000000,1.68,-5.18
712,23P530525,-581.567956,-653.956425,-425.870245,-528.90911,2022-07-21,2022-07-27,CAT,0.000000,0.57,-1.36,0.000000,2.22,-5.90
713,23P530525,-581.567956,-653.956425,-425.870245,-528.90911,2022-07-21,2022-07-27,TQQQ,0.000000,4.25,-5.50,0.000000,12.48,-7.40
714,23P530525,-581.567956,-653.956425,-425.870245,-528.90911,2022-07-21,2022-07-27,CHTR,-1.048626,1.97,-1.34,-1.431507,-0.64,-1.88


# Positions

In [23]:
import requests
import csv
import time
import json
import pandas as pd
import psycopg2
from psycopg2 import sql
import csv

BASE_URL = "https://srm-api.sterlingtradingtech.com/pts-risk"
USERNAME = "AOSScalp"
PASSWORD = "Scalp1"
mapping = {
        "3OB05285": "12P52852",
        "3OB05287": "14P52874",
        "3OB05259": "15P52595",
        "3OB05283": "16P52836",
        "3OB05286": "17P52867",
        "3OB05294": "18P529422",
        "3OB05295": "19P529523",
        "3OB05255": "1P525501",
        "3OB05292": "20P529221",
        "3OB05303": "21P530324",
        "3OB05306": "22P530624",
        "3OB05305": "23P530525",
        "3OB05311": "24P531126",
        "3OB05314": "25P531426",
        "3OB05315": "26P531527",
        "3OB05322": "27P532248",
        "3OB05323": "28P532349",
        "3OB05324": "29P532451",
        "3OB05258": "2P525802",
        "3OB05326": "30P532652",
        "3OB05327": "31P532753",
        "3OB05345": "32P534563",
        "3OB05346": "33P534612",
        "3OB05347": "34P534728",
        "3OB05360": "35P536046",
        "3OB05361": "36P536158",
        "3OB05362": "37P536299",
        "3OB05363": "38P536381",
        "3OB05364": "39P536458",
        "3OB05284": "3P528403",
        "3OB05365": "40P536571",
        "3OB05366": "41P536622",
        "3OB05367": "42P536787",
        "3OB05256": "4P525604",
        "3OB05257": "5P525705",
        "3OB05288": "6P528806",
        "3OB05260": "7P526007",
        "3OB05289": "8P528918",
        "3OB05261": "9P526109",
        "3OB05254": "SIPM5254"
    }

def get_access_token():
    url = f"{BASE_URL}/oauth/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": USERNAME,
        "client_secret": PASSWORD
    }
    response = requests.post(url, data=payload)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to obtain access token. Status code: {response.status_code}")

def call_api(endpoint, access_token, json_body):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    url = f"{BASE_URL}{endpoint}"
    
    response = requests.post(url, headers=headers, json=json_body)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed. Status code: {response.status_code}")

def positions_main(access_token):
    positions_json_body = ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"]
    
    positions_data = call_api("/service/account-positions", access_token, positions_json_body)
    return positions_data

def account_pos_run():
    access_token = get_access_token()
    print("Access token obtained:", access_token)

    positions = positions_main(access_token)

    rows = []
    for item in positions:
        account = item["account"]
        accountName = item["accountName"]
        for position in item["positions"]:
            row = {
                "account": account,
                "accountName": accountName,
                "symbol": position["symbol"],
                "pos": position["pos"]
            }
            rows.append(row)

    # Create a DataFrame from the rows list
    df = pd.DataFrame(rows)
    df['date'] = datetime.now()
    return df 

positions = account_pos_run()
positions

Access token obtained: 59e2413d-91af-485d-8bb9-df1280c715a6


,account,accountName,symbol,pos,date
0,3OB05311,3OB05311-24P,.AVGO 230915C910000,0,2023-05-30 13:26:01.856849
1,3OB05311,3OB05311-24P,.AVGO 250117P880000,0,2023-05-30 13:26:01.856849
2,3OB05311,3OB05311-24P,.NVDA 240621C770000,0,2023-05-30 13:26:01.856849
3,3OB05311,3OB05311-24P,.AI 240719C40000,0,2023-05-30 13:26:01.856849
4,3OB05311,3OB05311-24P,.AI 240719C15000,0,2023-05-30 13:26:01.856849
...,...,...,...,...,...
3081,3OB05292,3OB05292-20P,.ASML 230915P690000,0,2023-05-30 13:26:01.856849
3082,3OB05292,3OB05292-20P,.AVGO 230915C700000,0,2023-05-30 13:26:01.856849
3083,3OB05292,3OB05292-20P,.BKNG 230707P2460000,0,2023-05-30 13:26:01.856849
3084,3OB05292,3OB05292-20P,.XSD 240119C205000,0,2023-05-30 13:26:01.856849


# Greeks

In [26]:
import requests
import csv
import time
import json
import pandas as pd
import re
from datetime import datetime

BASE_URL = "https://srm-api.sterlingtradingtech.com/pts-risk"
USERNAME = "AOSScalp"
PASSWORD = "Scalp1"
mapping = {
    "3OB05285": "12P52852",
    "3OB05287": "14P52874",
    "3OB05259": "15P52595",
    "3OB05283": "16P52836",
    "3OB05286": "17P52867",
    "3OB05294": "18P529422",
    "3OB05295": "19P529523",
    "3OB05255": "1P525501",
    "3OB05292": "20P529221",
    "3OB05303": "21P530324",
    "3OB05306": "22P530624",
    "3OB05305": "23P530525",
    "3OB05311": "24P531126",
    "3OB05314": "25P531426",
    "3OB05315": "26P531527",
    "3OB05322": "27P532248",
    "3OB05323": "28P532349",
    "3OB05324": "29P532451",
    "3OB05258": "2P525802",
    "3OB05326": "30P532652",
    "3OB05327": "31P532753",
    "3OB05345": "32P534563",
    "3OB05346": "33P534612",
    "3OB05347": "34P534728",
    "3OB05360": "35P536046",
    "3OB05361": "36P536158",
    "3OB05362": "37P536299",
    "3OB05363": "38P536381",
    "3OB05364": "39P536458",
    "3OB05284": "3P528403",
    "3OB05365": "40P536571",
    "3OB05366": "41P536622",
    "3OB05367": "42P536787",
    "3OB05256": "4P525604",
    "3OB05257": "5P525705",
    "3OB05288": "6P528806",
    "3OB05260": "7P526007",
    "3OB05289": "8P528918",
    "3OB05261": "9P526109",
    "3OB05254": "SIPM5254"
}

def get_access_token():
    url = f"{BASE_URL}/oauth/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": USERNAME,
        "client_secret": PASSWORD
    }
    response = requests.post(url, data=payload)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to obtain access token. Status code: {response.status_code}")

def call_api(endpoint, access_token, json_body):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    url = f"{BASE_URL}{endpoint}"
    
    response = requests.post(url, headers=headers, json=json_body)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed. Status code: {response.status_code}")

def var_main(access_token):
    true = True
    var_json_body = {
        "accounts": ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"],
        "correlator": "correlator",
        "detailAnswer": true,
        "messageType": "Snapshot"
    }
    var_data = call_api("/service/var", access_token, var_json_body)
    return var_data

def greek_main(access_token, symbols_list, shockLevel, underlyingShock, volativityShock):
    greek_json_body = {
        "correlator": "test",
        "instruments": symbols_list,
        "messageType": "Snapshot",
        "shocks": [
            {
            "shockLevel": shockLevel,
            "underlyingShock": underlyingShock,
            "volatilityShock": volativityShock
            }
        ],
    }
    greek_data = call_api("/service/RiskGatewaySnapshot", access_token, greek_json_body)
    return greek_data

def positions_main(access_token):
    positions_json_body = ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"]
    
    # Replace 'your_endpoint' with the desired endpoint from the STT API documentation
    postitions_data = call_api("/service/account-positions", access_token, positions_json_body)
    print("API call result:", postitions_data)
    return postitions_data

def house_main(access_token):
    house_json_body = ["3OB05285", "3OB05287", "3OB05259", "3OB05283", "3OB05286", 
        "3OB05294", "3OB05295", "3OB05255", "3OB05292", "3OB05303", "3OB05306", 
        "3OB05305", "3OB05311", "3OB05314", "3OB05315", "3OB05322", "3OB05323", 
        "3OB05324", "3OB05258", "3OB05326", "3OB05327", "3OB05345", "3OB05346", 
        "3OB05347", "3OB05360", "3OB05361", "3OB05362", "3OB05363", "3OB05364", 
        "3OB05284", "3OB05365", "3OB05366", "3OB05367", "3OB05256", "3OB05257", 
        "3OB05288", "3OB05260", "3OB05289", "3OB05261", "3OB0525"]
    house_data = call_api("/service/house-policy", access_token, house_json_body)
    print("API call result:", house_data)
    return house_data

def clean_symbol(symbol):
    symbol = symbol.replace(" ", "")  # Remove all spaces
    if symbol.startswith("."):
        underlying = re.match(r'[^\d]+', symbol).group()
        underlying = underlying.replace(".", "")  # Remove the period from the underlying variable
        return symbol, underlying
    else:
        return None, symbol

def format_symbol(symbol):
    # Remove the decimal in the symbol
    symbol_formatted = symbol.replace('.', '')
    # Replace multiple spaces with a single space
    symbol_formatted = ' '.join(symbol_formatted.split())
    # Remove the space between the second set of numerical values and the "C" or "P"
    symbol_formatted = re.sub(r"([CP])(\s+)(\d+)", r"\1\3", symbol_formatted)
    # Add the period at the beginning
    symbol_formatted = '.' + symbol_formatted

    # assuming the underlying asset is determined by the part of the symbol before the first space
    underlying = symbol.split()[0]

    return symbol_formatted, underlying


def account_pos_run():
    access_token = get_access_token()
    print("Access token obtained:", access_token)

    positions = positions_main(access_token)

    rows = []
    for item in positions:
        account = item["account"]
        accountName = item["accountName"]
        for position in item["positions"]:
            row = {
                "account": account,
                "accountName": accountName,
                "symbol": position["symbol"],
                "pos": position["pos"]
            }
            rows.append(row)

    # Create a DataFrame from the rows list
    df = pd.DataFrame(rows)

    # Assuming your DataFrame is called df
    df['contract_name'], df['underlying'] = zip(*df['symbol'].apply(format_symbol))

    # Create a new DataFrame that contains the 'symbol', 'account', and 'pos' columns
    symbol_account_df = df[['symbol', 'account', 'pos']].drop_duplicates()

    # Create a list of unique symbols from the 'symbol' column
    symbols_list = df['symbol'].to_list()

    greek_data = greek_main(access_token, symbols_list, 'MarketShock', -100, 100)

    # Extract the 'results' list from the JSON data and create a DataFrame
    results_df = pd.DataFrame(greek_data['results'])

    # Merge the results_df with symbol_account_df on the 'symbol' column
    results_df_with_account_pos = results_df.merge(symbol_account_df, on='symbol')

    #results_df_with_account_pos.to_csv("greeks_df_with_account_pos.csv")
    return results_df_with_account_pos[
        [
            "account", "symbol", "pos", "optionPrice", "underlyingPrice",
            "shockedUnderlyingPrice", "volatility", "volatilityShock", "shockedVolatility",
            "theo", "delta", "gamma", "vega", "theta", "rho", "useClosePrice", "lastUpdate",
            "shockLevel"
        ]
    ]

def greeks_accounts_pos_df():
    # Run the script and write the initial data to the CSV file
    account_pos_df = account_pos_run()
    account_pos_df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Multiply each Greek value by the position and store the results in new columns
    account_pos_df['Theo Position'] = account_pos_df['theo'] * account_pos_df['pos']
    account_pos_df['Delta Position'] = account_pos_df['delta'] * account_pos_df['pos']
    account_pos_df['Gamma Position'] = account_pos_df['gamma'] * account_pos_df['pos']
    account_pos_df['Vega Position'] = account_pos_df['vega'] * account_pos_df['pos']
    account_pos_df['Theta Position'] = account_pos_df['theta'] * account_pos_df['pos']
    account_pos_df['Rho Position'] = account_pos_df['rho'] * account_pos_df['pos']

    new_column_order = [
        "timestamp", "lastUpdate", "account", "symbol", "pos",
        "optionPrice", "underlyingPrice", "shockedUnderlyingPrice",
        "volatility", "volatilityShock", "shockedVolatility",
        "theo", "delta", "gamma", "vega", "theta", "rho",
        "Theo Position", "Delta Position", "Gamma Position",
        "Vega Position", "Theta Position", "Rho Position"
    ]
    data_frame = account_pos_df[new_column_order]

    #Lower case the column names
    data_frame.columns = data_frame.columns.str.lower()
    return data_frame

greeks_accounts_pos_df = greeks_accounts_pos_df()
greeks_accounts_pos_df.to_csv("greeks.csv")

Access token obtained: 59e2413d-91af-485d-8bb9-df1280c715a6
API call result: [{'account': '3OB05311', 'accountName': '3OB05311-24P', 'positions': [{'symbol': '.AVGO 230915C910000', 'pos': 0}, {'symbol': '.AVGO 250117P880000', 'pos': 0}, {'symbol': '.NVDA 240621C770000', 'pos': 0}, {'symbol': '.AI 240719C40000', 'pos': 0}, {'symbol': '.AI 240719C15000', 'pos': 0}, {'symbol': '.ACN 240119C350000', 'pos': 0}, {'symbol': '.HIBS 231117C8000', 'pos': 0}, {'symbol': '.LRCX 230602C550000', 'pos': 0}, {'symbol': '.AVGO 240119C1150000', 'pos': 0}, {'symbol': '.SPY 250321C505000', 'pos': 0}, {'symbol': '.AVGO 240119C900000', 'pos': 0}, {'symbol': '.NVDA 230915C350000', 'pos': 0}, {'symbol': '.QQQ 230630C400000', 'pos': 0}, {'symbol': '.NVDA 240119C325000', 'pos': 0}, {'symbol': '.NVDA 240621P350000', 'pos': 0}, {'symbol': '.BKNG 230616P2450000', 'pos': 0}, {'symbol': '.MELI 230630P940000', 'pos': 0}, {'symbol': '.HIBS 231117C4000', 'pos': 0}, {'symbol': '.LRCX 230616C545000', 'pos': 0}, {'symbol'